# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [72]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()    #   u is a copy of given matrix
    L = np.eye(N)   #   L is an eye matrix
    for j in range(N-1):
        lam = np.eye(N) #   lam is the same as N
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [73]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [74]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [75]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [76]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [77]:
np.linalg.matrix_rank(a1)

6

In [78]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [79]:
def single_minor(matrix, i, j):
    return np.linalg.det(np.delete(np.delete(matrix, i, axis=0), j, axis=1))

def check_LU_cond(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[0]):
            if (single_minor(matrix, i, j) == 0):
                return False
    return True

print('Check for a matrix: {}'.format(check_LU_cond(a)))
print('Check for a1 matrix: {}'.format(check_LU_cond(a1)))

Check for a matrix: True
Check for a1 matrix: True


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [80]:
def diy_lu_piv(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()    #   u is a copy of given matrix
    L = np.eye(N)   #   L is an eye matrix
    permutation_matrix = np.eye(N)  #   Initialization for permutation matrix
    for j in range(N-1):
        max_index = np.abs(u[j, j:]).argmax() + j
        temporary_matrix = np.eye(N)
        temporary_matrix[j, j] = 0
        temporary_matrix[max_index, max_index] = 0
        temporary_matrix[j, max_index] = 1
        temporary_matrix[max_index, j] = 1
        permutation_matrix = np.dot(permutation_matrix, temporary_matrix)
        u[:,[j, max_index]] = u[:, [max_index, j]]  #   Swap finished
        lam = np.eye(N) #   lam is the same as N
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        #print('Iteration number {}: \n'.format(j))
        #print(u)

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, permutation_matrix

In [81]:
L, u, p = diy_lu_piv(a1)

print(L, "\n")
print(u, "\n")
print(p, '\n')

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u@p.T - a1)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.179 1.    0.    0.   ]
 [1.    1.231 1.294 1.605 1.    0.   ]
 [1.    1.25  1.375 2.007 2.076 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  2.220e-16 -9.247e-02 -4.755e-02 -1.920e-02]
 [ 0.000e+00 -1.344e-16 -3.565e-16  0.000e+00  1.841e-03  1.181e-03]
 [ 0.000e+00 -1.667e-16  2.941e-16 -1.567e-17 -4.337e-19 -1.233e-05]] 

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -2.220e-16  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+